# QuantumEdge Tutorial 3: Backtesting Portfolio Strategies

**Comprehensive Historical Performance Analysis and Strategy Comparison**

This notebook demonstrates how to backtest different portfolio optimization strategies using QuantumEdge's backtesting engine. We'll compare the historical performance of classical and quantum-inspired approaches across multiple market conditions.

## Overview

- **Buy and Hold**: Passive benchmark strategy
- **Rebalancing**: Periodic rebalancing to target weights
- **Mean-Variance**: Dynamic Markowitz optimization
- **VQE Strategy**: Quantum-inspired eigenportfolio rebalancing
- **QAOA Strategy**: Discrete asset selection optimization

---

## 1. Setup and Configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import json
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("QuantumEdge Backtesting Analysis")
print("=" * 40)

In [ ]:
# QuantumEdge API configuration
API_BASE_URL = "http://localhost:8000/api/v1"

def check_api_health():
    """Check if QuantumEdge API is running"""
    try:
        response = requests.get(f"{API_BASE_URL.replace('/api/v1', '')}/health")
        if response.status_code == 200:
            health_data = response.json()
            print("✅ QuantumEdge API is healthy")
            print(f"   Version: {health_data['version']}")
            print(f"   Services: {', '.join([k for k, v in health_data['services'].items() if v == 'healthy' or v == 'available'])}")
            return True
        else:
            print(f"❌ API health check failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Cannot connect to API: {e}")
        print("Make sure to run: cd /path/to/QuantumEdge && python -m src.api.main")
        return False

api_healthy = check_api_health()

## 2. Backtesting Configuration

Let's set up our backtesting parameters and universe of assets.

In [ ]:
# Backtesting configuration
BACKTEST_CONFIG = {
    'symbols': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'JPM', 'JNJ'],
    'start_date': '2020-01-01',
    'end_date': '2024-01-01',
    'initial_cash': 100000,
    'commission_rate': 0.001,  # 0.1% commission
    'min_commission': 1.0,
    'rebalance_frequency': 'monthly',
    'min_weight': 0.0,
    'max_weight': 0.4,
    'allow_short_selling': False,
    'benchmark_symbol': 'SPY',
    'risk_free_rate': 0.02
}

print("Backtesting Configuration:")
print("=" * 30)
for key, value in BACKTEST_CONFIG.items():
    if isinstance(value, list):
        print(f"{key}: {', '.join(value)}")
    else:
        print(f"{key}: {value}")

print(f"\nBacktest Period: {(pd.to_datetime(BACKTEST_CONFIG['end_date']) - pd.to_datetime(BACKTEST_CONFIG['start_date'])).days} days")
print(f"Number of Assets: {len(BACKTEST_CONFIG['symbols'])}")

## 3. Strategy Definitions

Let's define our portfolio strategies for backtesting comparison.

In [ ]:
def create_backtest_request(strategy_type, strategy_params=None):
    """Create a backtest request with common parameters"""
    
    base_request = {
        'strategy_type': strategy_type,
        'symbols': BACKTEST_CONFIG['symbols'],
        'start_date': BACKTEST_CONFIG['start_date'],
        'end_date': BACKTEST_CONFIG['end_date'],
        'initial_cash': BACKTEST_CONFIG['initial_cash'],
        'commission_rate': BACKTEST_CONFIG['commission_rate'],
        'min_commission': BACKTEST_CONFIG['min_commission'],
        'rebalance_frequency': BACKTEST_CONFIG['rebalance_frequency'],
        'min_weight': BACKTEST_CONFIG['min_weight'],
        'max_weight': BACKTEST_CONFIG['max_weight'],
        'allow_short_selling': BACKTEST_CONFIG['allow_short_selling'],
        'benchmark_symbol': BACKTEST_CONFIG['benchmark_symbol'],
        'risk_free_rate': BACKTEST_CONFIG['risk_free_rate']
    }
    
    # Add strategy-specific parameters
    if strategy_params:
        base_request.update(strategy_params)
    
    return base_request

# Define our strategy configurations
STRATEGIES = {
    'Buy and Hold': {
        'request': create_backtest_request('buy_and_hold', {
            'target_weights': {symbol: 1/len(BACKTEST_CONFIG['symbols']) 
                             for symbol in BACKTEST_CONFIG['symbols']}
        }),
        'description': 'Equal-weight buy and hold'
    },
    
    'Monthly Rebalancing': {
        'request': create_backtest_request('rebalancing', {
            'target_weights': {symbol: 1/len(BACKTEST_CONFIG['symbols']) 
                             for symbol in BACKTEST_CONFIG['symbols']}
        }),
        'description': 'Equal-weight with monthly rebalancing'
    },
    
    'Mean-Variance': {
        'request': create_backtest_request('mean_variance', {
            'lookback_period': 252,  # 1 year lookback
            'risk_aversion': 1.0
        }),
        'description': 'Dynamic mean-variance optimization'
    },
    
    'VQE Strategy': {
        'request': create_backtest_request('vqe', {
            'lookback_period': 126,  # 6 months lookback
            'depth': 3,
            'max_iterations': 50
        }),
        'description': 'Quantum eigenportfolio optimization'
    },
    
    'QAOA Strategy': {
        'request': create_backtest_request('qaoa', {
            'lookback_period': 126,  # 6 months lookback
            'num_layers': 2,
            'risk_aversion': 1.5,
            'cardinality_constraint': 5  # Select only 5 assets
        }),
        'description': 'Discrete asset selection with cardinality constraint'
    }
}

print("Defined Strategies:")
print("=" * 20)
for name, config in STRATEGIES.items():
    print(f"• {name}: {config['description']}")

## 4. Run Individual Backtests

Let's run each strategy individually to understand their performance characteristics.

In [ ]:
def run_single_backtest(strategy_name, request_data):
    """Run a single backtest strategy"""
    
    print(f"\nRunning {strategy_name}...")
    
    try:
        response = requests.post(f"{API_BASE_URL}/backtest/run", json=request_data)
        
        if response.status_code == 200:
            result = response.json()
            if result['success']:
                print(f"✅ {strategy_name} completed successfully")
                print(f"   Execution time: {result['execution_time']:.2f}s")
                
                if result.get('performance_metrics'):
                    metrics = result['performance_metrics']
                    print(f"   Total Return: {metrics['total_return']:.2%}")
                    print(f"   Sharpe Ratio: {metrics['sharpe_ratio']:.3f}")
                    print(f"   Max Drawdown: {metrics['max_drawdown']:.2%}")
                
                return result
            else:
                print(f"❌ {strategy_name} failed: {result.get('error_message', 'Unknown error')}")
                return None
        else:
            print(f"❌ API request failed: {response.status_code}")
            print(response.text)
            return None
            
    except Exception as e:
        print(f"❌ Error running {strategy_name}: {e}")
        return None

# Run all strategies
backtest_results = {}

if api_healthy:
    for strategy_name, config in STRATEGIES.items():
        result = run_single_backtest(strategy_name, config['request'])
        if result:
            backtest_results[strategy_name] = result
        
        # Rate limiting
        time.sleep(2)
    
    print(f"\n🎯 Successfully completed {len(backtest_results)} out of {len(STRATEGIES)} backtests")
else:
    print("❌ API not healthy - skipping backtests")
    backtest_results = {}

## 5. Performance Analysis

Now let's analyze and compare the performance of different strategies.

In [ ]:
if backtest_results:
    # Create performance comparison DataFrame
    performance_data = []
    
    for strategy_name, result in backtest_results.items():
        if result.get('performance_metrics'):
            metrics = result['performance_metrics']
            summary = result.get('summary', {})
            
            performance_data.append({
                'Strategy': strategy_name,
                'Total Return': metrics['total_return'],
                'CAGR': metrics['cagr'],
                'Volatility': metrics['annualized_volatility'],
                'Sharpe Ratio': metrics['sharpe_ratio'],
                'Sortino Ratio': metrics['sortino_ratio'],
                'Max Drawdown': metrics['max_drawdown'],
                'Calmar Ratio': metrics['calmar_ratio'],
                'VaR (95%)': metrics['var_95'],
                'CVaR (95%)': metrics['cvar_95'],
                'Final Value': summary.get('final_value', 0),
                'Total Commissions': summary.get('total_commissions', 0),
                'Num Rebalances': summary.get('num_rebalances', 0)
            })
    
    performance_df = pd.DataFrame(performance_data)
    
    print("\nPerformance Comparison:")
    print("=" * 50)
    
    # Display key metrics
    key_metrics = ['Strategy', 'Total Return', 'CAGR', 'Volatility', 'Sharpe Ratio', 'Max Drawdown']
    display_df = performance_df[key_metrics].copy()
    
    # Format percentages
    for col in ['Total Return', 'CAGR', 'Volatility', 'Max Drawdown']:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.2%}")
    
    # Format ratios
    display_df['Sharpe Ratio'] = display_df['Sharpe Ratio'].apply(lambda x: f"{x:.3f}")
    
    print(display_df.to_string(index=False))
else:
    print("❌ No backtest results available for analysis")
    performance_df = pd.DataFrame()

## 6. Visualization and Charts

In [ ]:
if backtest_results and len(performance_df) > 0:
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Total Returns Comparison
    strategies = performance_df['Strategy']
    total_returns = performance_df['Total Return']
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(strategies)))
    bars1 = axes[0,0].bar(strategies, total_returns, color=colors, alpha=0.7)
    axes[0,0].set_title('Total Returns Comparison', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('Total Return')
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[0,0].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars1, total_returns):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                      f'{value:.1%}', ha='center', va='bottom', fontsize=10)
    
    # 2. Risk-Adjusted Returns (Sharpe Ratio)
    sharpe_ratios = performance_df['Sharpe Ratio']
    bars2 = axes[0,1].bar(strategies, sharpe_ratios, color=colors, alpha=0.7)
    axes[0,1].set_title('Sharpe Ratio Comparison', fontsize=14, fontweight='bold')
    axes[0,1].set_ylabel('Sharpe Ratio')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars2, sharpe_ratios):
        axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                      f'{value:.3f}', ha='center', va='bottom', fontsize=10)
    
    # 3. Risk vs Return Scatter
    volatilities = performance_df['Volatility']
    cagrs = performance_df['CAGR']
    
    scatter = axes[1,0].scatter(volatilities, cagrs, s=200, c=sharpe_ratios, 
                               cmap='viridis', alpha=0.7, edgecolors='black')
    axes[1,0].set_title('Risk vs Return Profile', fontsize=14, fontweight='bold')
    axes[1,0].set_xlabel('Volatility (Risk)')
    axes[1,0].set_ylabel('CAGR')
    axes[1,0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1%}'))
    axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,0].grid(True, alpha=0.3)
    
    # Add strategy labels
    for i, strategy in enumerate(strategies):
        axes[1,0].annotate(strategy, (volatilities.iloc[i], cagrs.iloc[i]),
                          xytext=(5, 5), textcoords='offset points',
                          fontsize=9, alpha=0.8)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=axes[1,0])
    cbar.set_label('Sharpe Ratio')
    
    # 4. Drawdown Comparison
    max_drawdowns = performance_df['Max Drawdown'].abs()  # Make positive for display
    bars4 = axes[1,1].bar(strategies, max_drawdowns, color='red', alpha=0.6)
    axes[1,1].set_title('Maximum Drawdown Comparison', fontsize=14, fontweight='bold')
    axes[1,1].set_ylabel('Maximum Drawdown')
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,1].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars4, max_drawdowns):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                      f'{value:.1%}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available for visualization")

## 7. Portfolio Evolution Over Time

Let's examine how portfolio values evolved throughout the backtest period.

In [ ]:
if backtest_results:
    # Plot portfolio value evolution
    plt.figure(figsize=(15, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(backtest_results)))
    
    for i, (strategy_name, result) in enumerate(backtest_results.items()):
        if result.get('portfolio_values'):
            portfolio_values = result['portfolio_values']
            dates = [pd.to_datetime(pv['date']) for pv in portfolio_values]
            values = [pv['value'] for pv in portfolio_values]
            
            plt.plot(dates, values, label=strategy_name, color=colors[i], linewidth=2)
    
    # Add benchmark if available
    if len(backtest_results) > 0:
        first_result = list(backtest_results.values())[0]
        if first_result.get('benchmark_values'):
            benchmark_values = first_result['benchmark_values']
            benchmark_dates = [pd.to_datetime(bv['date']) for bv in benchmark_values]
            benchmark_vals = [bv['value'] for bv in benchmark_values]
            
            plt.plot(benchmark_dates, benchmark_vals, label='SPY Benchmark', 
                    color='black', linestyle='--', linewidth=2, alpha=0.7)
    
    plt.title('Portfolio Value Evolution Over Time', fontsize=16, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calculate and display cumulative returns
    print("\nCumulative Returns Analysis:")
    print("=" * 30)
    
    for strategy_name, result in backtest_results.items():
        if result.get('portfolio_values'):
            initial_value = result['portfolio_values'][0]['value']
            final_value = result['portfolio_values'][-1]['value']
            total_return = (final_value / initial_value) - 1
            
            print(f"{strategy_name}:")
            print(f"  Initial: ${initial_value:,.2f}")
            print(f"  Final: ${final_value:,.2f}")
            print(f"  Total Return: {total_return:.2%}")
            print()
else:
    print("❌ No portfolio value data available")

## 8. Risk Analysis and Drawdown Charts

In [ ]:
if len(performance_df) > 0:
    # Create risk analysis charts
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Risk Metrics Comparison
    risk_metrics = ['Volatility', 'VaR (95%)', 'CVaR (95%)']
    x = np.arange(len(performance_df))
    width = 0.25
    
    for i, metric in enumerate(risk_metrics):
        values = performance_df[metric].abs()  # Make positive for display
        axes[0,0].bar(x + i*width, values, width, label=metric, alpha=0.7)
    
    axes[0,0].set_title('Risk Metrics Comparison', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('Risk Level')
    axes[0,0].set_xlabel('Strategy')
    axes[0,0].set_xticks(x + width)
    axes[0,0].set_xticklabels(performance_df['Strategy'], rotation=45)
    axes[0,0].legend()
    axes[0,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Risk-Adjusted Performance Ratios
    ratios = ['Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio']
    x = np.arange(len(performance_df))
    
    for i, ratio in enumerate(ratios):
        values = performance_df[ratio]
        axes[0,1].bar(x + i*width, values, width, label=ratio, alpha=0.7)
    
    axes[0,1].set_title('Risk-Adjusted Performance Ratios', fontsize=14, fontweight='bold')
    axes[0,1].set_ylabel('Ratio Value')
    axes[0,1].set_xlabel('Strategy')
    axes[0,1].set_xticks(x + width)
    axes[0,1].set_xticklabels(performance_df['Strategy'], rotation=45)
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Transaction Costs Analysis
    commissions = performance_df['Total Commissions']
    rebalances = performance_df['Num Rebalances']
    
    ax3_twin = axes[1,0].twinx()
    
    bars1 = axes[1,0].bar(x - width/2, commissions, width, label='Total Commissions', 
                         color='red', alpha=0.6)
    bars2 = ax3_twin.bar(x + width/2, rebalances, width, label='Num Rebalances', 
                        color='blue', alpha=0.6)
    
    axes[1,0].set_title('Transaction Costs and Rebalancing', fontsize=14, fontweight='bold')
    axes[1,0].set_ylabel('Total Commissions ($)', color='red')
    ax3_twin.set_ylabel('Number of Rebalances', color='blue')
    axes[1,0].set_xlabel('Strategy')
    axes[1,0].set_xticks(x)
    axes[1,0].set_xticklabels(performance_df['Strategy'], rotation=45)
    axes[1,0].tick_params(axis='y', labelcolor='red')
    ax3_twin.tick_params(axis='y', labelcolor='blue')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Final Portfolio Values
    final_values = performance_df['Final Value']
    colors = ['green' if v > BACKTEST_CONFIG['initial_cash'] else 'red' 
              for v in final_values]
    
    bars4 = axes[1,1].bar(performance_df['Strategy'], final_values, 
                         color=colors, alpha=0.7)
    
    # Add initial value line
    axes[1,1].axhline(y=BACKTEST_CONFIG['initial_cash'], color='black', 
                     linestyle='--', label=f"Initial Value (${BACKTEST_CONFIG['initial_cash']:,})")
    
    axes[1,1].set_title('Final Portfolio Values', fontsize=14, fontweight='bold')
    axes[1,1].set_ylabel('Portfolio Value ($)')
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'${y:,.0f}'))
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars4, final_values):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
                      f'${value:,.0f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No performance data available for risk analysis")

## 9. Strategy Comparison Summary

Let's run a comprehensive strategy comparison using the QuantumEdge comparison endpoint.

In [ ]:
if api_healthy and len(STRATEGIES) > 1:
    # Prepare comparison request
    comparison_request = {
        'strategies': [config['request'] for config in STRATEGIES.values()],
        'strategy_names': list(STRATEGIES.keys())
    }
    
    print("Running comprehensive strategy comparison...")
    
    try:
        response = requests.post(f"{API_BASE_URL}/backtest/compare", 
                               json=comparison_request, 
                               timeout=300)  # 5 minute timeout
        
        if response.status_code == 200:
            comparison_result = response.json()
            
            if comparison_result['success']:
                print(f"✅ Comparison completed in {comparison_result['execution_time']:.2f}s")
                
                # Display performance comparison table
                if comparison_result.get('performance_comparison'):
                    print("\nStrategy Performance Comparison:")
                    print("=" * 50)
                    
                    comparison_df = pd.DataFrame(comparison_result['performance_comparison'])
                    print(comparison_df.to_string(index=False))
                
                # Plot comparison chart
                if comparison_result.get('portfolio_values'):
                    plt.figure(figsize=(15, 8))
                    
                    colors = plt.cm.tab10(np.linspace(0, 1, len(comparison_result['portfolio_values'])))
                    
                    for i, (strategy_name, portfolio_data) in enumerate(comparison_result['portfolio_values'].items()):
                        dates = [pd.to_datetime(pv['date']) for pv in portfolio_data]
                        values = [pv['value'] for pv in portfolio_data]
                        
                        plt.plot(dates, values, label=strategy_name, 
                                color=colors[i], linewidth=2)
                    
                    plt.title('Strategy Comparison - Portfolio Evolution', 
                             fontsize=16, fontweight='bold')
                    plt.xlabel('Date')
                    plt.ylabel('Portfolio Value ($)')
                    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                    plt.grid(True, alpha=0.3)
                    plt.tight_layout()
                    plt.show()
            else:
                print(f"❌ Comparison failed: {comparison_result.get('error_message')}")
        else:
            print(f"❌ Comparison request failed: {response.status_code}")
            print(response.text)
            
    except requests.exceptions.Timeout:
        print("⏰ Comparison request timed out - this is normal for complex backtests")
    except Exception as e:
        print(f"❌ Error running comparison: {e}")
else:
    print("❌ Cannot run comparison - API not healthy or insufficient strategies")

## 10. Statistical Analysis and Insights

In [ ]:
if len(performance_df) > 0:
    # Calculate additional insights
    insights = []
    
    # Best performing strategy
    best_return_idx = performance_df['Total Return'].idxmax()
    best_return_strategy = performance_df.loc[best_return_idx, 'Strategy']
    best_return_value = performance_df.loc[best_return_idx, 'Total Return']
    insights.append(f"🏆 Highest Returns: {best_return_strategy} ({best_return_value:.2%})")
    
    # Best risk-adjusted performance
    best_sharpe_idx = performance_df['Sharpe Ratio'].idxmax()
    best_sharpe_strategy = performance_df.loc[best_sharpe_idx, 'Strategy']
    best_sharpe_value = performance_df.loc[best_sharpe_idx, 'Sharpe Ratio']
    insights.append(f"📈 Best Risk-Adjusted: {best_sharpe_strategy} (Sharpe: {best_sharpe_value:.3f})")
    
    # Lowest drawdown
    lowest_dd_idx = performance_df['Max Drawdown'].abs().idxmin()
    lowest_dd_strategy = performance_df.loc[lowest_dd_idx, 'Strategy']
    lowest_dd_value = performance_df.loc[lowest_dd_idx, 'Max Drawdown']
    insights.append(f"🛡️ Lowest Drawdown: {lowest_dd_strategy} ({lowest_dd_value:.2%})")
    
    # Most stable (lowest volatility)
    lowest_vol_idx = performance_df['Volatility'].idxmin()
    lowest_vol_strategy = performance_df.loc[lowest_vol_idx, 'Strategy']
    lowest_vol_value = performance_df.loc[lowest_vol_idx, 'Volatility']
    insights.append(f"🔒 Most Stable: {lowest_vol_strategy} (Vol: {lowest_vol_value:.2%})")
    
    # Cost efficiency
    lowest_cost_idx = performance_df['Total Commissions'].idxmin()
    lowest_cost_strategy = performance_df.loc[lowest_cost_idx, 'Strategy']
    lowest_cost_value = performance_df.loc[lowest_cost_idx, 'Total Commissions']
    insights.append(f"💰 Lowest Costs: {lowest_cost_strategy} (${lowest_cost_value:.2f})")
    
    print("\n" + "="*60)
    print("BACKTESTING INSIGHTS AND ANALYSIS")
    print("="*60)
    
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")
    
    # Statistical significance tests
    print("\n" + "="*60)
    print("STATISTICAL ANALYSIS")
    print("="*60)
    
    # Return statistics
    returns_stats = performance_df[['Strategy', 'Total Return', 'CAGR', 'Volatility', 'Sharpe Ratio']]
    print("\nReturn Statistics Summary:")
    print(f"Mean Total Return: {performance_df['Total Return'].mean():.2%}")
    print(f"Std Total Return: {performance_df['Total Return'].std():.2%}")
    print(f"Mean Sharpe Ratio: {performance_df['Sharpe Ratio'].mean():.3f}")
    print(f"Std Sharpe Ratio: {performance_df['Sharpe Ratio'].std():.3f}")
    
    # Risk statistics
    print(f"\nRisk Statistics Summary:")
    print(f"Mean Volatility: {performance_df['Volatility'].mean():.2%}")
    print(f"Mean Max Drawdown: {performance_df['Max Drawdown'].mean():.2%}")
    print(f"Mean VaR (95%): {performance_df['VaR (95%)'].mean():.2%}")
    
    # Correlation analysis
    numeric_cols = ['Total Return', 'CAGR', 'Volatility', 'Sharpe Ratio', 'Max Drawdown']
    correlation_matrix = performance_df[numeric_cols].corr()
    
    print("\nMetric Correlations:")
    print(correlation_matrix.round(3))
else:
    print("❌ No performance data available for statistical analysis")

## 11. Recommendations and Conclusions

In [ ]:
print("\n" + "="*70)
print("STRATEGY RECOMMENDATIONS AND CONCLUSIONS")
print("="*70)

recommendations = [
    "📊 **Portfolio Construction Insights**:",
    "   • Buy-and-hold strategies minimize transaction costs but may lag in volatile markets",
    "   • Regular rebalancing can improve risk-adjusted returns but increases costs",
    "   • Dynamic optimization strategies adapt to changing market conditions",
    "   • Quantum-inspired approaches may offer unique diversification benefits",
    "",
    "🎯 **Strategy Selection Guidelines**:",
    "   • Conservative investors: Focus on lowest volatility and drawdown strategies",
    "   • Growth-oriented investors: Prioritize total return and CAGR metrics",
    "   • Risk-adjusted optimization: Select strategies with highest Sharpe ratios",
    "   • Cost-conscious investors: Consider transaction costs and rebalancing frequency",
    "",
    "⚡ **Implementation Considerations**:",
    "   • Quantum strategies may require more computational resources but offer novel approaches",
    "   • Classical mean-variance remains highly effective for most portfolios",
    "   • Combine multiple strategies for portfolio-of-portfolios approach",
    "   • Regular backtesting helps validate strategy performance over time",
    "",
    "🔬 **Research Opportunities**:",
    "   • Test strategies across different market regimes (bull, bear, sideways)",
    "   • Explore sector-specific and geographic diversification effects",
    "   • Investigate alternative risk measures and optimization objectives",
    "   • Study the impact of transaction costs on strategy selection"
]

for recommendation in recommendations:
    print(recommendation)

print("\n" + "="*70)
print("NEXT STEPS")
print("="*70)

next_steps = [
    "1. 📈 Analyze advanced risk metrics with Tutorial 4 (Advanced Objectives)",
    "2. 🔍 Deep dive into risk analysis with Tutorial 5 (Risk Analysis)",
    "3. 🎛️ Experiment with different rebalancing frequencies and lookback periods",
    "4. 🌍 Test strategies on different asset classes and markets",
    "5. 📊 Implement real-time monitoring for live portfolio management",
    "6. 🤖 Explore machine learning integration for enhanced strategy selection"
]

for step in next_steps:
    print(step)

print(f"\n🎉 Backtesting analysis complete! Analyzed {len(backtest_results)} strategies over {BACKTEST_CONFIG['start_date']} to {BACKTEST_CONFIG['end_date']}")

## Summary

This comprehensive backtesting analysis demonstrates the power of QuantumEdge's portfolio optimization platform:

### **Key Findings**

1. **Strategy Performance**: Different optimization approaches show varying performance characteristics across risk and return metrics

2. **Risk Management**: Advanced risk measures (CVaR, drawdown, volatility) help identify strategies suitable for different risk preferences

3. **Cost Impact**: Transaction costs and rebalancing frequency significantly affect net returns, especially for high-turnover strategies

4. **Quantum vs Classical**: Both quantum-inspired and classical approaches offer unique advantages depending on portfolio constraints and objectives

### **Practical Applications**

- **Asset Managers**: Use backtesting to validate strategy performance before deployment
- **Risk Managers**: Analyze drawdown patterns and tail risk characteristics
- **Researchers**: Compare novel optimization techniques against established benchmarks
- **Individual Investors**: Select strategies aligned with personal risk tolerance and objectives

### **Technical Achievement**

This notebook showcases QuantumEdge's ability to:
- Handle multiple optimization strategies simultaneously
- Process large-scale historical backtests efficiently
- Provide comprehensive performance analytics
- Support both classical and quantum-inspired approaches

The backtesting framework provides the foundation for evidence-based portfolio management decisions, enabling users to select optimization strategies based on historical performance rather than theoretical assumptions.

---

**Continue your QuantumEdge journey:**
- `04_advanced_objectives.ipynb` - Deep dive into CVaR, Calmar, and Sortino optimization
- `05_risk_analysis.ipynb` - Comprehensive risk metric analysis and monitoring